# Model building for Dog Breed Classifier

### Here we use tensorflow transfer learning technique. We are using ' Inception V3 ' model for giving features and at last we are building feed forward networks for identifying the breed based on features.

### Loading all modules

In [1]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model,load_model
from glob import glob
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense,Activation,GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
#Execute below 2 lines if you are using NVIDIA GPU or else comment it
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],True)

### We load model without top for our purpose

In [2]:
inception_model =InceptionV3(include_top=False,input_shape=[299,299,3])

### As the transfer learning models are trained on huge data we need not train it again.

In [3]:
for layer in inception_model.layers:
    layer.trainable = False

### Importent here is to note that I had used GlobalAveragePooling2D() instead of Flatten(). Reason here is simple:
### <li>I wanted to deploy the model. So to keep its size small I used GlobalAverage instead of Flatten.
###  Because, GlobalAverage drops many parameters where Flatten takes all of them. Since Im having transfer learning model, Its ok to loose some of the information and it doesnt effect much.
#### (Purely on experiment basis)

In [4]:
x =GlobalAveragePooling2D()(inception_model.output)

x = Dense(30)(x)

pred = Activation('softmax')(x)

### Joining tranfer learning model with Dense layers

In [5]:
model = Model(inputs=inception_model.input,outputs=pred)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

## Generation of training and test data
### Here we increase the training data by data augumentation process

In [6]:
train_datagen =  ImageDataGenerator(rescale=1./255,
                                   shear_range=0.15,
                                   rotation_range=20,
                                   width_shift_range=0.1,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   height_shift_range=0.1
                                   )
test_datagen = ImageDataGenerator(rescale=1./255)


train_set = train_datagen.flow_from_directory('data/train',
                                              target_size=(299,299),
                                              batch_size=16,
                                              class_mode='categorical'
                                             )

test_set = train_datagen.flow_from_directory('data/test',
                                              target_size=(299,299),
                                              batch_size=16,
                                              class_mode='categorical'
                                             )

Found 2311 images belonging to 30 classes.
Found 254 images belonging to 30 classes.


### Creating the checkpoint of the best model

In [7]:
checkpoint = ModelCheckpoint('smart_model_mobile.h5',monitor='val_accuracy',verbose=1,save_best_only=True)
callback = [checkpoint]

### Compiling of model with Adam optimizer and crossentropy loss

In [8]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

### Training process

In [9]:
history = model.fit(train_set,epochs=30,steps_per_epoch=len(train_set),validation_data=test_set,verbose=1,callbacks=callback)

Epoch 1/30
145/145 [==============================] - ETA: 0s - loss: 0.9112 - accuracy: 0.8204
Epoch 00001: val_accuracy improved from -inf to 0.92520, saving model to smart_model_mobile.h5
145/145 [==============================] - 42s 288ms/step - loss: 0.9112 - accuracy: 0.8204 - val_loss: 0.2954 - val_accuracy: 0.9252
Epoch 2/30
145/145 [==============================] - ETA: 0s - loss: 0.1980 - accuracy: 0.9528
Epoch 00002: val_accuracy improved from 0.92520 to 0.92913, saving model to smart_model_mobile.h5
145/145 [==============================] - 38s 261ms/step - loss: 0.1980 - accuracy: 0.9528 - val_loss: 0.2465 - val_accuracy: 0.9291
Epoch 3/30
145/145 [==============================] - ETA: 0s - loss: 0.1438 - accuracy: 0.9589
Epoch 00003: val_accuracy did not improve from 0.92913
145/145 [==============================] - 37s 258ms/step - loss: 0.1438 - accuracy: 0.9589 - val_loss: 0.2538 - val_accuracy: 0.9134
Epoch 4/30
145/145 [==============================] - ETA: 0s 

## At the end we got 99% training accuracy and 94% accuracy on test data
#  <center>Thank You